Note:
This notebook submit jobs to the clusters. For each simulation, the algorithm will calculate the geometric features. These features were then further used for migratory modes classification.

In order to use this notebook, some of the paths might need to be adapted.

In [1]:
import numpy as np
import sys
from datetime import datetime
import subprocess
import os

import pipe_util2

In [2]:
def parameter_txt(param_folder, script_py_file, 
                  data_folder, feature_folder, figure_folder,
                  repeats, iterations, timepoints
                 ):
    """
    repeats, iterations - int
    timepoints - list(int)
    
    return - number of jobs
    """
    param_folder = pipe_util2.folder_verify(param_folder)
    param_folder = pipe_util2.create_folder(param_folder)
    
    count = 0
    i = 0
    while i<repeats:
        
        j = 0
        while j<iterations:
            
            k = 0
            while k<len(timepoints):
                
                file_num = str(count)
                f = open(param_folder+file_num+".p", "w")
                
                line = "python %s %s %s %s %s %s %s"%(
                    script_py_file, 
                    data_folder, feature_folder, figure_folder,
                    i, j, timepoints[k]
                )
                f.write(line)
                f.close()
                
                count = count+1
                k = k+1
            j = j+1
        i = i+1
        
    return repeats*iterations*len(timepoints)

In [3]:
def sbatch_file_polarity_sim(
    param_folder, scr_folder, name = None):
    """
    output_folder = string, folder for storing the .o file. Usually the same as script folder
    """
    
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y-%H%M")
    scr_folder = pipe_util2.create_folder(scr_folder)
    param_folder = pipe_util2.folder_verify(param_folder)
    
    if name:
        ofile_name = scr_folder + dt_string + "geo_calc_" + name + ".o"
        sfile_name = scr_folder + dt_string + "geo_calc_" + name + ".sh"
        
    else:
        ofile_name = scr_folder + dt_string + "geo_calc" + ".o"
        sfile_name = scr_folder + dt_string + "geo_calc" + ".sh"
    
    ofile_name = os.path.abspath(ofile_name)
    sfile_name = os.path.abspath(sfile_name)
    
    contents = """#!/bin/bash
#SBATCH --account=huijing
#SBATCH --partition=dept_cpu
#SBATCH --cpus-per-task=1
#SBATCH --mem=3G
#SBATCH --output=%s%s_%s.o
#SBATCH --job-name=potts_scan

eval "$(conda shell.bash hook)"
module load anaconda
source activate mg1
echo 'env activated'

bash %s$SLURM_ARRAY_TASK_ID.p
    """%(scr_folder, "%A", "%a", param_folder)
    s = open(sfile_name, "w")
    s.write(contents)
    s.close()
    
    return sfile_name

In [4]:
sim_name_list = [
    "20250129_b0_3_force_fpp_scan",
    "20250129_b0_6_force_fpp_scan",
    "20250129_b0_force_fpp_scan",
    "20250129_b1_force_fpp_scan"
]

In [5]:
script_py_file = "./feature_calc_6_simple_corr_edit.py"
script_py_file = os.path.abspath(script_py_file)

analysis_folder = "20250129-update-msmm-model-analysis"
pipe_util2.create_folder(analysis_folder)
analysis_folder = os.path.abspath(analysis_folder)

cmd_list = []
for sim_name in sim_name_list:
    
    data_folder = "./20250129-update-msmm-model-sim-data/CC3D_data_%s"%sim_name
    data_folder = os.path.abspath(data_folder)
    
    figure_folder = analysis_folder+"/"+"%s_calc6_figures/"%sim_name
    feature_folder = analysis_folder+"/"+"%s_calc6_features/"%sim_name
    pipe_util2.create_folder(figure_folder)
    pipe_util2.create_folder(feature_folder)


    param_folder = analysis_folder+"/"+"%s_param/"%sim_name
    scr_folder = analysis_folder+"/"+"%s_script/"%sim_name

    repeats = 50
    iterations = 231
    timepoints = [90,270]
    jobmax_num = parameter_txt(param_folder, script_py_file, 
                               data_folder, feature_folder, figure_folder,
                               repeats, iterations, timepoints
                              )

    sbatch_file = sbatch_file_polarity_sim(
        param_folder, scr_folder, name = None)

    batch_size = 800
    if jobmax_num<batch_size:
        batch_size = jobmax_num+1
    batch_size = "%"+"%s"%batch_size
    cmd = "sbatch --array=[0-%s]%s %s"%(jobmax_num, batch_size, sbatch_file)
    #subprocess.call(cmd, shell=True)
    cmd_list.append(cmd)


20250129-update-msmm-model-analysis/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-analysis/20250129_b0_3_force_fpp_scan_calc6_figures/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-analysis/20250129_b0_3_force_fpp_scan_calc6_features/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-analysis/20250129_b0_3_force_fpp_scan_param/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-analysis/20250129_b0_3_force_fpp_scan_script/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/20250129-update-msmm-model-analysis/20250129_b0_6_force_fpp_scan_calc6_figures/ folder is freshly created. 

/net/capricorn/home/xing/huijing/Migration/20250128_Supplements++/2025012

In [7]:
for cmd in cmd_list:
    subprocess.call(cmd, shell=True)

Submitted batch job 45097418
Submitted batch job 45097419
Submitted batch job 45097420
Submitted batch job 45097421
